In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
import tmm_core as tmm
import numpy as np
import matplotlib.pyplot as plt
import nidn
import torch

In [ ]:
nidn.set_log_level("info")

In [ ]:
cfg = nidn.load_default_cfg()
materials = [
             "titanium_oxide", 
             "tantalum_pentoxide",
             "silicon_nitride"
            ]
cfg.N_freq = 50 # number of frequency points
thicknesses = [
               0.38,
               0.1,
               1.0
              ] # thicknesses
# wavelengths
lam_l = [
         np.linspace(0.4, 0.5, cfg.N_freq),
         np.linspace(0.2, 0.4, cfg.N_freq),
         np.linspace(1.0, 2.0, cfg.N_freq)
        ]

cfg.N_layers = 1
cfg.FDTD_min_gridpoints_per_unit_magnitude = 150
iters = np.array([
                  1000,
                  500,
                  2500
                 ]) * cfg.FDTD_min_gridpoints_per_unit_magnitude / 50
print(iters)
cfg.FDTD_pulse_type = 'continuous'
cfg.FDTD_source_type = 'line'

In [ ]:
R_tmm,T_tmm,R_rcwa,T_rcwa,R_fdtd,T_fdtd = {}, {}, {}, {}, {}, {}

for idx, material in enumerate(materials):
    print(f"Computing material: {material}")
    # Set up NIDN
    cfg.PER_LAYER_THICKNESS=[thicknesses[idx]]
    cfg.physical_wavelength_range[0]=lam_l[idx][0] * 1e-6 # converting to micrometers
    cfg.physical_wavelength_range[1]=lam_l[idx][-1] *1e-6
    cfg.target_frequencies = nidn.compute_target_frequencies(
        cfg.physical_wavelength_range[0],
        cfg.physical_wavelength_range[1],
        cfg.N_freq,
        cfg.freq_distribution
    )

    # Init eps_grid
    eps_grid = torch.zeros(cfg.Nx,cfg.Ny,cfg.N_layers,cfg.N_freq,dtype=torch.cfloat)
    layer_builder = nidn.LayerBuilder(cfg)
    eps_grid[:,:,0,:] = layer_builder.build_uniform_layer(material)        

    # Set up TMM
    f_in = r'../nidn/materials/data/' + material + '.csv'
    nk = np.loadtxt(f_in, skiprows=1)
    d = [tmm.inf, thicknesses[idx], tmm.inf]
    n_mat = np.interp(lam_l[idx], nk[:, 0], nk[:, 1])
    k_mat = np.interp(lam_l[idx], nk[:, 0], nk[:, 2])

    # Compute TMM spectra    
    R,T = [], []
    for lam, n, k in zip(lam_l[idx], n_mat, k_mat):
        n_l = [1.0, n + 1.0j * k, 1.0]

        Res = tmm.coh_tmm('s', n_l, d, 0.0, lam)

        R.append(Res['R'])
        T.append(Res['T'])
    R_tmm[material] = R
    T_tmm[material] = T

    # Compute RCWA spectra
    cfg.solver = "TRCWA"
    R, T = nidn.compute_spectrum(eps_grid, cfg)
    R_rcwa[material] = R
    T_rcwa[material] = T

    # Compute FDTD spectra
    cfg.FDTD_niter = int(iters[idx])
    cfg.solver = "FDTD"
    R, T = nidn.compute_spectrum(eps_grid, cfg)
    R_fdtd[material] = R
    T_fdtd[material] = T


In [ ]:
for idx, material in enumerate(materials):
    print(f"Plotting material: {material}")

    # Recomputing frequencies just to doublecheck
    cfg.physical_wavelength_range[0]=lam_l[idx][0] * 1e-6
    cfg.physical_wavelength_range[1]=lam_l[idx][-1] * 1e-6
    cfg.target_frequencies = nidn.compute_target_frequencies(
        cfg.physical_wavelength_range[0],
        cfg.physical_wavelength_range[1],
        cfg.N_freq,
        cfg.freq_distribution
    )
    lam_fdtd,_ = nidn.get_frequency_points(cfg)
    lam_fdtd = lam_fdtd*10**6

    fig, axs = plt.subplots(1, 3,figsize=(12,3),dpi=150,facecolor='white')

    plt.subplot(1, 3, 1)
    plt.plot(lam_l[idx], R_tmm[material], label='TMM')
    plt.plot(lam_fdtd, R_rcwa[material], label='RCWA')
    plt.plot(lam_fdtd, R_fdtd[material], label='FDTD')
    plt.xlabel('lam')
    plt.ylabel('Reflectance')
    plt.legend()
    plt.title(material)
    plt.ylim([0,1])

    plt.subplot(1, 3, 2)
    plt.plot(lam_l[idx], T_tmm[material], label='TMM')
    plt.plot(lam_fdtd, T_rcwa[material], label='RCWA')
    plt.plot(lam_fdtd, T_fdtd[material], label='FDTD')
    plt.xlabel('lam')
    plt.ylabel('Transmittance')
    plt.legend()
    plt.title(material)
    plt.ylim([-0.1,1.1])

    plt.subplot(1, 3, 3)
    plt.plot(lam_l[idx], 1 - np.array(R_tmm[material]) - np.array(T_tmm[material]), label='TMM')
    plt.plot(lam_fdtd, 1 - np.array(R_rcwa[material]) - np.array(T_rcwa[material]), label='RCWA')
    plt.plot(lam_fdtd, 1 - np.array(R_fdtd[material]) - np.array(T_fdtd[material]), label='FDTD')
    plt.xlabel('lam')
    plt.ylabel('Absorbance')
    plt.legend()
    plt.title(material)
    plt.ylim([-0.1,1.1])
    plt.savefig(f'../results/{material}.png')